In [20]:
import datetime
import numpy as np
import pandas as pd
import pickle
import os.path
#pd.core.common.is_list_like = pd.api.types.is_list_like
#import pandas_datareader as web

from sklearn import preprocessing
#from keras.models import Sequential
#from keras.models import load_model

In [21]:
Xcolumns = pickle.load(open("max_Xcolumns.sav","rb"))
columns_array = []

In [22]:
for index in range(len(Xcolumns)):
    columns_array.append(Xcolumns[index][:4])
columns_unique_array = list(set(columns_array))
columns_unique_array.append(1321)

In [23]:
for code in columns_unique_array:
    print(code)
    
    path = "importETF/" + str(code) + ".csv"
    
    if os.path.exists(path):
        temp_df = pd.read_csv(path,engine = "python" ,encoding="utf8")
    else:
        error_array.append(code)
        continue
        
    new_df = pd.DataFrame()
    new_df["Date"] = temp_df["Date"]
    
    #始値
    new_df["Open"] = 0
    
    for dateIndex in temp_df.index:
        
        #当日の始値
        openValue = temp_df.at[dateIndex,"Open"]
        
        new_df.at[dateIndex,"High"] = temp_df.at[dateIndex,"High"] - openValue
        new_df.at[dateIndex,"Low"] = temp_df.at[dateIndex,"Low"] - openValue
        new_df.at[dateIndex,"Close"] = temp_df.at[dateIndex,"Close"] - openValue
        
        if dateIndex != 0:
            new_df.at[dateIndex,"Volume"] = temp_df.at[dateIndex,"Volume"] - temp_df.at[dateIndex-1,"Volume"]
            new_df.at[dateIndex,"Open"] = openValue - temp_df.at[dateIndex-1,"Close"]
            
        else:
            new_df.at[0,"Volume"] = 0
    
    #csvファイル書き出し
    new_df.to_csv("PredictStockDataDif/" + str(code) + "_dif.csv")

1323
1625
1552
1623
1689
1626
1308
1326
1672
1541
1325
1695
1329
1629
1620
1546
1619
1621
1632
1633
1670
1696
1330
1542
1622
1683
1627
1343
1615
1545
1319
1543
1617
1682
1348
1697
1612
1624
1324
1551
1310
1628
1313
1630
1678
1699
1681
1698
1613
1631
1679
1306
1309
1349
1671
1680
1690
1346
1321
1327
1320
1344
1540
1305
1345
1618
1547
1322
1311
1677
1550
1321


In [ ]:
ETF_df = pd.DataFrame()

startDate = "2010/01/01"

df = pd.read_csv("importETF.csv",encoding="utf8")

for code in df["code"]:
    
    if code not in error_array:
        code = str(code)
        
        temp = pd.DataFrame()
        temp = pd.read_csv("PredictStockDataDif/" + code + "_dif.csv",encoding="utf8")
        
        if code == "1305":
            #初回のみETF_dfにindexを設定
            ETF_df["Date"] = temp["Date"]
            ETF_df = ETF_df[ETF_df["Date"] > startDate]
            ETF_df = ETF_df.set_index("Date")
            
        if code =="1321":
            for dateIndex in range(0,len(temp.index)-1):
                tempDate = temp.at[dateIndex,"Date"]
                
                if tempDate <= startDate:
                    continue
                    
                tempClose = temp.at[dateIndex+1,"Close"]
                if tempClose >= 20:
                    ETF_df.at[tempDate,"nextDay_HighLow"] = 1
                elif tempClose >= 0:
                    ETF_df.at[tempDate,"nextDay_HighLow"] = 0
                elif tempClose > -20:
                    ETF_df.at[tempDate,"nextDay_HighLow"] = 0
                else:
                    ETF_df.at[tempDate,"nextDay_HighLow"] = -1
                    
        for dateIndex in temp.index:
            tempDate = temp.at[dateIndex,"Date"]
            
            if tempDate <= startDate:
                continue
                
            ETF_df.at[tempDate,code + "Open"] = temp.at[dateIndex,"Open"]
            ETF_df.at[tempDate,code + "High"] = temp.at[dateIndex,"High"]
            ETF_df.at[tempDate,code + "Low"] = temp.at[dateIndex,"Low"]
            ETF_df.at[tempDate,code + "Close"] = temp.at[dateIndex,"Close"]
            ETF_df.at[tempDate,code + "Volume"] = temp.at[dateIndex,"Volume"]/10000
            

In [ ]:
path = "importETF/1321.csv"

if os.path.exists(path):
    nikkei1321_df = pd.read_csv(path,engine = "python" ,encoding="utf8")
else:
    error_array.append(code)


In [52]:
df = pd.read_csv('nikkei_2001_2018.csv',encoding='SHIFT-JIS')

In [53]:
model_open = Sequential()
model_open = load_model('model_open')

model_close = Sequential()
model_close = load_model('model_close')

In [54]:
#modelのinput_shapeの要素数を取得
input_shape = model_open.layers[0].get_input_at(0).get_shape().as_list()[1]

In [55]:
scaler = preprocessing.MinMaxScaler()
scaler.fit([[7200],[24790]])
scaler.fit([[1],[15]])


MinMaxScaler(copy=True, feature_range=(0, 1))

In [56]:
df.head()

,日付,始値,高値,安値,終値,出来高,終値調整値
0,2001-07-13,1,12490,12300,2,259430,12310
1,2001-07-16,3,12400,12250,2,99740,12330
2,2001-07-17,5,12170,12100,6,93150,12120
3,2001-07-18,7,12150,11830,6,165330,11840
4,2001-07-19,9,11990,11860,10,620220,11920


In [57]:
input_data = input()

end_index = df.query('日付 == "' + str(input_data) + '"').index
start_index = end_index - input_shape

df = df[start_index.values[0]:end_index.values[0]]

2001-08-18


In [58]:
open_date = df.loc[:, ['始値']]
close_date = df.loc[:, ['終値']]

In [59]:
open_date = np.array(open_date)
close_date = np.array(close_date)

In [60]:
open_date = scaler.transform(open_date)
close_date = scaler.transform(close_date)

open_date = open_date.reshape(1,input_shape,1)
close_date = close_date.reshape(1,input_shape,1)

In [61]:
predicted_open = model_open.predict(open_date)
predicted_close = model_close.predict(close_date)
print(predicted_open)
print(predicted_close)

[[ 0.57892853]]
[[ 0.54372144]]


In [62]:
if predicted_open <= predicted_close:
    print('上昇')
else:
    print('降下')

降下
